### Домашнее задание "Рекомендации на основе содержания"
Преподаватель: Алексей Кузьмин
- Использовать dataset MovieLens
- Построить рекомендации (регрессия, предсказываем оценку) на фичах:
    - TF-IDF на тегах и жанрах
    - Средние оценки (+ median, variance, etc.) пользователя и фильма
- Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
links = pd.read_csv('./data/links.csv')
movies = pd.read_csv('./data/movies.csv')
ratings = pd.read_csv('./data/ratings.csv')
tags = pd.read_csv('./data/tags.csv')

### 1. Собираем очевидные фичи

In [3]:
# Собираем все фильмы и оценки
df = ratings.join(movies.set_index('movieId'), on='movieId')

In [4]:
# Оставляем только те фильмы, по которым есть теги
movies_with_tags = tags.movieId.unique()
df = df[df.movieId.isin(movies_with_tags)]

In [5]:
# Агрегируем по фильму и считаем среднюю оценку и количество оценок
df_agg = df.groupby(by='movieId').agg(['mean', 'count', 'var']).rating.reset_index()

# Считаем статистики по количеству оценок
mean_num_ratigs = df_agg['count'].mean()
min_num_ratigs = df_agg['count'].min()
max_num_ratigs = df_agg['count'].max()

# Взвешиваем рейтинг по нормированному количеству оценок
df_agg['weighted_rating'] = df_agg.apply(lambda x: x['mean'] * (x['count'] - mean_num_ratigs) / (max_num_ratigs - min_num_ratigs), axis=1)

In [6]:
# Добавляем жанры
df_agg = df_agg.merge(movies, on='movieId', how='left')

In [7]:
# Собираем все теги по каждому фильму
grouped_tags = tags.groupby(by='movieId')

film_tags = {}
for key, value in grouped_tags.groups.items():
    film_tags[key] = tags.loc[value.values].tag.tolist()

In [8]:
# Добавляем тэги
df_agg['tags'] = df_agg.apply(lambda x: film_tags[x.movieId], axis=1)

In [9]:
# Причесываем жанры
df_agg.genres = df_agg.apply(lambda x: x.genres.split('|'), axis=1)

In [10]:
df_agg['full_tags'] = df_agg.apply(lambda x: x.genres + x.tags, axis=1)

In [11]:
df_agg['full_tags'] = df_agg.apply(lambda x: ' '.join(x.full_tags), axis=1)

In [12]:
# Готово. Красиво.
df_agg.drop(columns=['genres', 'tags'], inplace=True)
df_agg.head()

,movieId,mean,count,var,weighted_rating,title,full_tags
0,1,3.920930,215,0.696990,2.198677,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
1,2,3.431818,110,0.777419,0.825805,Jumanji (1995),Adventure Children Fantasy fantasy magic board...
2,3,3.259615,52,1.112651,0.207972,Grumpier Old Men (1995),Comedy Romance moldy old
3,5,3.071429,49,0.822917,0.167873,Father of the Bride Part II (1995),Comedy pregnancy remake
4,7,3.185185,54,0.955625,0.222645,Sabrina (1995),Comedy Romance remake


### 2. Обработаем текстовые фичи

In [13]:
all_tags = df_agg.full_tags.tolist()

In [14]:
countvect = CountVectorizer()
tfidf = TfidfTransformer()

In [15]:
all_films = countvect.fit_transform(all_tags)
all_films_tfidf = tfidf.fit_transform(all_films)

In [16]:
all_films_tfidf = all_films_tfidf.toarray()

In [17]:
tfidf_data = pd.DataFrame(all_films_tfidf, index=df_agg.movieId, columns=['t_'+str(i) for i in range(len(all_films_tfidf[0]))])

In [18]:
df_full = df_agg.merge(tfidf_data, on='movieId')

Здесь у нас готовы данные по всем фильмам. Дальше берем конкретного пользователя и строим для него персональные рекомендации регрессионной моделью.

### 3. Выбираем пользователя и делаем train-test split

In [19]:
df_full_to_train = df_full.drop(columns=['title', 'full_tags']).set_index('movieId')

In [20]:
df_full_to_train = df_full_to_train.fillna(value=df_full_to_train['var'].mean())

In [21]:
# Мы усекли датафрейм только теми фильмами, у которых есть теги.
# Следовательно рекомендовать мы сможем только юзерам, которые оставляли теги. Найдем таких.
users = ratings[ratings.movieId.isin(movies_with_tags)].userId.unique()
users[:10]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [22]:
# Вот сколько фильмов оценил наш тестовый пользователь. Берем его.
ratings.userId.value_counts()[1]

232

In [23]:
user_films = ratings[(ratings.userId == 1) & ratings.movieId.isin(movies_with_tags)].drop(columns='timestamp')

In [24]:
user_films = user_films.join(df_full_to_train, on='movieId')

In [25]:
X = user_films.drop(columns=['userId', 'rating']).set_index('movieId')
y = user_films.loc[:, user_films.columns.isin(['movieId', 'rating'])].set_index('movieId')

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [31]:
lm = Lasso(alpha=0.001, max_iter=1000, normalize=True, tol=0.0001)

In [32]:
lm.fit(X_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [33]:
#Наша средняя ошибка составляет
mean_squared_error(y_test, lm.predict(X_test))

1.0391748177647273

In [35]:
# Проверим на кросс-валидации
cross_val_score(lm, X, y, scoring='neg_mean_squared_error', cv=5).mean()

-0.8154798854998084

In [36]:
lm.fit(X, y)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

### 4. Рекомендация

In [37]:
user_films.movieId.unique()

array([   1,    3,   47,   50,  101,  110,  216,  223,  235,  260,  296,
        316,  349,  356,  457,  480,  500,  527,  543,  552,  590,  592,
        593,  596,  608,  648,  673,  733,  736,  780,  919,  923,  940,
        943,  954, 1025, 1029, 1030, 1032, 1042, 1080, 1089, 1090, 1097,
       1136, 1196, 1197, 1198, 1206, 1208, 1210, 1213, 1214, 1219, 1220,
       1222, 1224, 1240, 1258, 1270, 1278, 1282, 1291, 1348, 1377, 1396,
       1408, 1500, 1573, 1580, 1617, 1625, 1732, 1777, 1954, 2028, 2054,
       2058, 2078, 2115, 2116, 2139, 2268, 2291, 2329, 2387, 2470, 2502,
       2529, 2571, 2616, 2628, 2640, 2641, 2648, 2692, 2700, 2716, 2761,
       2797, 2872, 2959, 2987, 3052, 3147, 3176, 3247, 3273, 3386, 3489,
       3527, 3578, 3671, 3793])

In [38]:
# Для начала из нашего подготовленного списка фильмов, фильмы, которые смотрел наш пользователь.
X_rec = df_full_to_train.iloc[~df_full_to_train.index.isin(user_films.movieId.unique())]

In [39]:
X_rec['user_predicted_score'] = lm.predict(X_rec)

/home/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
personal_recomandations = X_rec[['user_predicted_score', 'mean', 'weighted_rating']].sort_values('user_predicted_score', ascending=False)

In [48]:
# Вот последнее, что оценил пользователь
ratings[ratings.userId == 1].merge(movies.set_index('movieId'), on='movieId').sort_values('timestamp', ascending=False)[:10]

,userId,movieId,rating,timestamp,title,genres
161,1,2492,4.0,965719662,20 Dates (1998),Comedy|Romance
119,1,2012,4.0,964984176,Back to the Future Part III (1990),Adventure|Comedy|Sci-Fi|Western
160,1,2478,4.0,964984169,¡Three Amigos! (1986),Comedy|Western
31,1,553,5.0,964984153,Tombstone (1993),Action|Drama|Western
95,1,1445,3.0,964984112,McHale's Navy (1997),Comedy|War
9,1,157,5.0,964984100,Canadian Bacon (1995),Comedy|War
42,1,780,3.0,964984086,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller
201,1,3053,5.0,964984086,"Messenger: The Story of Joan of Arc, The (1999)",Drama|War
90,1,1298,5.0,964984086,Pink Floyd: The Wall (1982),Drama|Musical
214,1,3448,5.0,964984054,"Good Morning, Vietnam (1987)",Comedy|Drama|War


In [49]:
# Фильмы про войну, вестерныб комедии - ОК.

In [50]:
personal_recomandations.merge(movies.set_index('movieId'), on='movieId')[:20]

,user_predicted_score,mean,weighted_rating,title,genres
movieId,,,,,
6650,7.019803,3.333333,-0.285292,Kind Hearts and Coronets (1949),Comedy|Drama
25825,6.384292,3.375000,-0.278568,Fury (1936),Drama|Film-Noir
5088,6.295797,5.000000,-0.458426,"Going Places (Valseuses, Les) (1974)",Comedy|Crime|Drama
4454,6.287986,5.000000,-0.458426,More (1998),Animation|Drama|Sci-Fi|IMAX
40491,6.184801,5.000000,-0.458426,"Match Factory Girl, The (Tulitikkutehtaan tytt...",Comedy|Drama
6732,6.181567,3.750000,-0.332386,"Hello, Dolly! (1969)",Comedy|Musical|Romance
152711,6.179354,5.000000,-0.458426,Who Killed Chea Vichea? (2010),Documentary
247,5.996276,3.928571,-0.120644,Heavenly Creatures (1994),Crime|Drama
7834,5.992365,2.833333,-0.242498,After the Thin Man (1936),Comedy|Crime|Mystery|Romance


In [51]:
# Вроде неплохо
# Готовим пайплайн, проводим A/B тесты